In [3]:
# !pip install pandas
# !pip install pyarrow
# !pip install geopandas
# !pip install matplotlib
# !pip install seaborn
# !pip install pyproj
# !pip install geopy
# !pip install "geopy[aiohttp]"
# !pip install tqdm
import logging
import os
from random import randint
from time import sleep

import geopandas as gpd
import pandas as pd
from geopy.adapters import AioHTTPAdapter
from geopy.exc import GeocoderServiceError, GeocoderTimedOut
from geopy.geocoders import Nominatim
from shapely.geometry import Polygon
from tqdm import tqdm


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
def geopy_client_geo_reverse(geolocator, coordinate):
    location = geolocator.reverse(coordinate, timeout = 3, language="en")
    country = location.raw.get('address').get('country')
    print("Coordinates " + coordinate , " Country: " + country , " file name: " + file_name)
    return country
    
def convert_polygon_to_country(poly_items, file_name):
    return_list= [] 
    index = 0
    print(str(len(poly_items)) + " rows have been given initially to the script ")
    for poly_item in tqdm(poly_items):
        longitude,latitude = poly_item.bounds[:2]
        my_user_agent = 'UTwente_Managing_Big_Data_project_{}'.format(randint(10000,99999))
        geolocator = Nominatim(user_agent = my_user_agent)
        coordinate = str(latitude) +","+ str(longitude)
        try:
            country = geopy_client_geo_reverse(geolocator, coordinate)
            return_list.append(country)
            index = index +1
        except GeocoderTimedOut:
            logging.info('TIMED OUT: GeocoderTimedOut: Retrying...')
            sleep(randint(1*100,5*100)/100)
            country = geopy_client_geo_reverse(geolocator, coordinate)
            return_list.append(country)
        except GeocoderServiceError as e:
            logging.info('CONNECTION REFUSED: GeocoderServiceError encountered.')
            logging.error(e)
            return None
        except Exception as e:
            logging.info('ERROR: Terminating due to exception {}'.format(e))
            return None    
    print(str(index) + " have been processed ")
    print(str(len(poly_items)) + " have been given initially to the script ")
    return return_list

In [ ]:
%%time
for subdir, dirs, files in os.walk('dataset'):
    for file in files:
        if file.endswith('shp'):
            file_name = os.path.join(subdir,file)
            print("Reading now from " + file_name )
            shape_file = gpd.read_file(file_name)
           
            new_file_name = 'countries_' + file.split('.')[0] +'.parquet'
            new_file_path = os.path.join(subdir,new_file_name)
            
            print("Create new parquet files ..............")
            
            parquet_file = pd.DataFrame(columns=['quadkey','avg_d_kbps','avg_u_kbps','avg_lat_ms','tests','devices', 'tile'])
            parquet_file.quadkey = shape_file.quadkey
            parquet_file.avg_d_kbps = shape_file.avg_d_kbps
            parquet_file.avg_u_kbps = shape_file.avg_u_kbps
            parquet_file.avg_lat_ms = shape_file.avg_lat_ms
            parquet_file.tests = shape_file.tests
            parquet_file.devices = shape_file.devices

            print("Converting polygons to countries ..............")
            parquet_file.tile = convert_polygon_to_country(shape_file.geometry,file_name)
            # parquet_file = pd.read_parquet('dataset/2019-q1/countries_gps_fixed_tiles.parquet', engine='pyarrow')
            print("Writing now to " + new_file_path)
            parquet_file.to_parquet(new_file_path)
            print("Finished ...................................................")
            
     

In [ ]:
for subdir, dirs, files in os.walk('dataset'):
    for file in files:
        if file.startswith('countries'):
            file_name = os.path.join(subdir,file)
            parquet_file = pd.read_parquet(file_name, engine='pyarrow')
            display(parquet_file)

In [ ]:
shape_file = gpd.read_file('dataset/2019-q1/gps_fixed_tiles.shp')
print(len(shape_file.index))
# parquet_file = pd.read_parquet('dataset/2019-q1/2019-01-01_performance_fixed_tiles.parquet', engine='pyarrow')
# display(shape_file)
# display(parquet_file)
# parquet_file = pd.DataFrame(columns=['quadkey','country'])
# parquet_file.quadkey = shape_file.quadkey
# parquet_file.country = convert_polygon_to_country(shape_file.geometry)
# parquet_file.to_parquet('dataset/2019-q1/countries_gps_fixed_tiles.parquet')
# parquet_file = pd.read_parquet('dataset/2019-q1/countries_gps_fixed_tiles.parquet', engine='pyarrow')
# display(parquet_file)

In [ ]:
print(shape_file.iloc[4788])

In [ ]:
total_row_count = 0
for subdir, dirs, files in os.walk('dataset'):
    for file in files:
        if file.endswith('parquet'):
            file_df = pd.read_parquet(os.path.join(subdir,file))
            total_row_count += len(file_df.index)
            print(file + "containts ", str(len(file_df.index)) + " rows")

print("total count ", str(total_row_count))

In [ ]:
%%time
q1_2019 = 'dataset/2019-q1/2019-01-01_performance_fixed_tiles.parquet' 
q1_df = pd.read_parquet(q1_2019, engine='pyarrow', columns=['quadkey', 'tile'])
pd.set_option('display.max_colwidth', 3000)
display(q1_df)

In [ ]:
%%time
q1_shape_file = gpd.read_file('dataset/2019-q1/gps_fixed_tiles.shp', driver='shapely')
q1_shape_file.head()
display(q1_shape_file)

In [ ]:
q1_parquet_file1 = q1_shape_file.iloc[:2]
q1_parquet_file =  q1_parquet_file1[['quadkey','geometry']].copy()
q1_parquet_file1.head()
q1_parquet_file.to_file('dataset/2019-q1/2019-01-01_performance_fixed_tiles_countries.shp')

In [ ]:
parquet_file = gpd.read_file('dataset/2019-q1/2019-01-01_performance_fixed_tiles_countries.shp')
parquet_file.head()

In [ ]:
def replace_polygons_with_countries_in_given_parquet_file(file_path):
    parquet_file = pd.read_parquet(file_path, engine='pyarrow')
    parquet_file_len = len(parquet_file.index)
    print("length " + str(parquet_file_len))
    index = 0
    while index < parquet_file_len:
        parquet_file = pd.read_parquet(file_path, engine='pyarrow')
        temp_index = index
        for inner_index in range(index, temp_index + 100):
            parquet_file.loc[inner_index,'geometry'] = convert_polygon_to_country(parquet_file.loc[inner_index,'geometry'])
        parquet_file.to_parquet('dataset/2019-q1/2019-01-01_performance_fixed_tiles_countries.parquet', index=False)
        index =  inner_index   
    # list_of_polygons = q1_csv_file['geometry']

replace_polygons_with_countries_in_given_parquet_file('dataset/2019-q1/2019-01-01_performance_fixed_tiles_countries.parquet')    